In [1]:
import json
import numpy as np
import pandas as pd
import lightgbm as lgb
from dask import dataframe as dd

In [2]:
transactions = dd.read_csv('data/transactions_train.csv')[["customer_id", "article_id"]]
customer_purchase_number = transactions.groupby("customer_id").size().to_frame("prod_number").reset_index()
transactions = transactions.merge(customer_purchase_number, on="customer_id", how="inner")

c_ids = transactions.customer_id.unique()
number_of_customer = len(c_ids)
customer_encoding = {c_id: i for i, c_id in enumerate(c_ids)}
p_ids = transactions.article_id.unique()
number_of_products = len(p_ids)
product_encoding = {p_id: i for i, p_id in enumerate(p_ids)}
with open('model_data/customer_id_encoding.json', 'w') as fp:
    json.dump(customer_encoding, fp)

with open('model_data/product_id_encoding.json', 'w') as fp:
    json.dump(product_encoding, fp)

transactions.customer_id = transactions.customer_id.map(customer_encoding)
transactions.article_id = transactions.article_id.map(product_encoding)
p_ids = list(product_encoding.values())
del c_ids, product_encoding, customer_encoding, customer_purchase_number

In [3]:
train, test = transactions.random_split([0.9, 0.1], random_state=43)
df = test.merge(train[["customer_id"]], on=["customer_id"], how="outer", indicator=True)
train = dd.concat([train, df[(df._merge == 'left_only') |  (df.prod_number == 1)][["customer_id", "article_id"]]], axis=0, ignore_index=True, interleave_partitions=True, ignore_order=True)
df = df[(df._merge == 'both') &( df.prod_number > 1)][["customer_id"]].drop_duplicates()
test = test.merge(df, how="inner", on="customer_id")
train = train.drop(["prod_number"], axis=1)
test = test.drop(["prod_number"], axis=1)
cols = ["customer_id", "article_id"]
train[cols] = train[cols].applymap(np.int32)
test[cols] = test[cols].applymap(np.int32)
train = train.compute()
test = test.compute()
del transactions, df

In [4]:
train.sort_values(by=["customer_id"], inplace=True)
test.sort_values(by=["customer_id"], inplace=True)
q_train = train.groupby("customer_id").size().to_frame("prod_number").reset_index()
q_test = train.groupby("customer_id").size().to_frame("prod_number").reset_index() 
train["article_id"] = pd.Series(train.article_id, dtype="category")
test["article_id"] = pd.Series(test.article_id, dtype="category")

In [5]:
train["label"] = 1
test["label"] = 1
train = train.drop(["customer_id"], axis=1)
test = test.drop(["customer_id"], axis=1)
train = dd.from_pandas(train, npartitions= 8)
test = dd.from_pandas(test, npartitions= 4)

In [6]:
gbm = lgb.LGBMRanker()

In [11]:
gbm.fit(
    train, train.label, group=q_train.prod_number, early_stopping_rounds=10,
    eval_set = [(test, test.label)], eval_group = [q_test.prod_number], verbose=10,
    eval_at = [3,7,10]
)

/Users/limon/opt/anaconda3/envs/tf/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/limon/opt/anaconda3/envs/tf/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
